### Import modules

In [15]:
import json
import pandas as pd
import requests
import numpy as np
import re as re

Please specify industry to upgrade and code (ht6) that best represents the industry:

In [16]:
industry = 'baterias'
main_ht6_code = 850730

### Helpful functions

In [17]:
def get_countries(code):
    '''
    Gets all the countries that export the product refered by a single code, 
    helpful to do the exact number of queries needed.
    
    Input: A code (string)
    
    Output: list of countries (integers)
    '''
    url = "http://comtrade.un.org/api/get?max=500&type=C&freq=A&px=HS&ps=now&r=all&p=0&rg=2&cc=" + str(code)
    f = requests.get(url, timeout=300)
    x = json.loads(f.text)
    df_prueba = pd.DataFrame(x['dataset'])
    df_prueba = df_prueba[['pfCode', 'period', 'aggrLevel', 'rgDesc', 'rtTitle', 'ptTitle', 'rtCode',
                       'cmdCode', 'cmdDescE', 'TradeQuantity', 'TradeValue']]
    paises = df_prueba['rtCode'].unique().tolist()
    list_paises = [paises[i:i + 5] for i in range(0, len(paises), 5)]
    return list_paises

def query(url):
    '''
    Executes a single query from the UN Comtrade API, converts json object
    into pandas dataframe.
    
    Input: A url (string)
    
    Output: Dataframe with the output of the query (pandas dataframe).
    '''
    f = requests.get(url, timeout=300)
    x = json.loads(f.text)
    df = pd.DataFrame(x['dataset'])
    df = df[['pfCode', 'period', 'aggrLevel', 'rgDesc', 'rtTitle', 'ptTitle',
                       'cmdCode', 'cmdDescE', 'TradeQuantity', 'TradeValue']]
    return df

def query_v2(url):
    '''
    Executes a single query from the UN Comtrade API, converts json object
    into pandas dataframe, modification to include NUMERICAL codes por countries.
    
    Input: A url (string)
    
    Output: Dataframe with the output of the query (pandas dataframe).
    '''
    f = requests.get(url, timeout=300)
    x = json.loads(f.text)
    df = pd.DataFrame(x['dataset'])
    df = df[['pfCode', 'period', 'aggrLevel', 'rgDesc', 'rtTitle', 'ptTitle',
                       'cmdCode', 'cmdDescE', 'TradeQuantity', 'TradeValue', 'rtCode']]
    return df  

Initialize dataframe called 'base'. Initialize lists (helpful to start at the place the last run ended, do not repeat, just run once) and load codes from csv in the directory.

In [18]:
data = []
base = pd.DataFrame(data, columns=['pfCode', 'period', 'aggrLevel', 'rgDesc', 'rtTitle', 'ptTitle',
                       'cmdCode', 'cmdDescE', 'TradeQuantity', 'TradeValue'])

string = 'codes_' + industry + '.csv'
codes_all = pd.read_csv(string)
codes_all = codes_all.astype(str)
codes_all = codes_all['code'].to_list()
lst_check, lst_code = [], [0,0]

### Download data

Repeat the next chunk until all the codes are covered, you can check this down by checking the base dataframe.

In [19]:
for cod in codes_all:
    if cod not in lst_code:
        print(cod)
        for sublist in get_countries(cod):
            string = '' + str(sublist[0])
            for con in sublist[1:]:
                string = string + '%2C' + str(con)
            comb = str(cod) + str(string)
            if comb not in lst_check:
                url = "http://comtrade.un.org/api/get?max=50000&type=C&freq=A&px=HS&ps=2020&r=" + str(string) + "&p=all&rg=2&cc=" + str(cod)
                print(url)
                row = query(url)
                base = pd.concat([base, row])
                lst_check.append(comb)
                print(url, cod, comb)
        lst_code.append(cod)

253090
http://comtrade.un.org/api/get?max=50000&type=C&freq=A&px=HS&ps=2020&r=31%2C32%2C36%2C40%2C51&p=all&rg=2&cc=253090
http://comtrade.un.org/api/get?max=50000&type=C&freq=A&px=HS&ps=2020&r=31%2C32%2C36%2C40%2C51&p=all&rg=2&cc=253090 253090 25309031%2C32%2C36%2C40%2C51
http://comtrade.un.org/api/get?max=50000&type=C&freq=A&px=HS&ps=2020&r=56%2C68%2C70%2C72%2C76&p=all&rg=2&cc=253090
http://comtrade.un.org/api/get?max=50000&type=C&freq=A&px=HS&ps=2020&r=56%2C68%2C70%2C72%2C76&p=all&rg=2&cc=253090 253090 25309056%2C68%2C70%2C72%2C76
http://comtrade.un.org/api/get?max=50000&type=C&freq=A&px=HS&ps=2020&r=104%2C112%2C156%2C170%2C191&p=all&rg=2&cc=253090
http://comtrade.un.org/api/get?max=50000&type=C&freq=A&px=HS&ps=2020&r=104%2C112%2C156%2C170%2C191&p=all&rg=2&cc=253090 253090 253090104%2C112%2C156%2C170%2C191
http://comtrade.un.org/api/get?max=50000&type=C&freq=A&px=HS&ps=2020&r=196%2C203%2C208%2C222%2C233&p=all&rg=2&cc=253090
http://comtrade.un.org/api/get?max=50000&type=C&freq=A&px=HS

http://comtrade.un.org/api/get?max=50000&type=C&freq=A&px=HS&ps=2020&r=490%2C516%2C528%2C586%2C616&p=all&rg=2&cc=250410 250410 250410490%2C516%2C528%2C586%2C616
http://comtrade.un.org/api/get?max=50000&type=C&freq=A&px=HS&ps=2020&r=642%2C688%2C699%2C702%2C704&p=all&rg=2&cc=250410
http://comtrade.un.org/api/get?max=50000&type=C&freq=A&px=HS&ps=2020&r=642%2C688%2C699%2C702%2C704&p=all&rg=2&cc=250410 250410 250410642%2C688%2C699%2C702%2C704
http://comtrade.un.org/api/get?max=50000&type=C&freq=A&px=HS&ps=2020&r=705%2C710%2C716%2C724%2C752&p=all&rg=2&cc=250410
http://comtrade.un.org/api/get?max=50000&type=C&freq=A&px=HS&ps=2020&r=705%2C710%2C716%2C724%2C752&p=all&rg=2&cc=250410 250410 250410705%2C710%2C716%2C724%2C752
http://comtrade.un.org/api/get?max=50000&type=C&freq=A&px=HS&ps=2020&r=757%2C764%2C792%2C804%2C826&p=all&rg=2&cc=250410
http://comtrade.un.org/api/get?max=50000&type=C&freq=A&px=HS&ps=2020&r=757%2C764%2C792%2C804%2C826&p=all&rg=2&cc=250410 250410 250410757%2C764%2C792%2C804%2C

http://comtrade.un.org/api/get?max=50000&type=C&freq=A&px=HS&ps=2020&r=440%2C458%2C490%2C528%2C554&p=all&rg=2&cc=280519 280519 280519440%2C458%2C490%2C528%2C554
http://comtrade.un.org/api/get?max=50000&type=C&freq=A&px=HS&ps=2020&r=579%2C608%2C616%2C699%2C702&p=all&rg=2&cc=280519
http://comtrade.un.org/api/get?max=50000&type=C&freq=A&px=HS&ps=2020&r=579%2C608%2C616%2C699%2C702&p=all&rg=2&cc=280519 280519 280519579%2C608%2C616%2C699%2C702
http://comtrade.un.org/api/get?max=50000&type=C&freq=A&px=HS&ps=2020&r=705%2C710%2C724%2C752%2C757&p=all&rg=2&cc=280519
http://comtrade.un.org/api/get?max=50000&type=C&freq=A&px=HS&ps=2020&r=705%2C710%2C724%2C752%2C757&p=all&rg=2&cc=280519 280519 280519705%2C710%2C724%2C752%2C757
http://comtrade.un.org/api/get?max=50000&type=C&freq=A&px=HS&ps=2020&r=764%2C792%2C826%2C842%2C818&p=all&rg=2&cc=280519
http://comtrade.un.org/api/get?max=50000&type=C&freq=A&px=HS&ps=2020&r=764%2C792%2C826%2C842%2C818&p=all&rg=2&cc=280519 280519 280519764%2C792%2C826%2C842%2C

http://comtrade.un.org/api/get?max=50000&type=C&freq=A&px=HS&ps=2020&r=440%2C458%2C528%2C554%2C616&p=all&rg=2&cc=282520 282520 282520440%2C458%2C528%2C554%2C616
http://comtrade.un.org/api/get?max=50000&type=C&freq=A&px=HS&ps=2020&r=642%2C699%2C702%2C710%2C724&p=all&rg=2&cc=282520


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

Save to a csv just to have a back-up between runs of the code.

In [20]:
string = 'base_' + industry + '.csv'
base.to_csv (string, index = False, header=True)
base

### Add stage of the value chain and save in a csv.

In [22]:
#Pedro's file
base = pd.read_csv(string)
stages = pd.read_excel (r'Descripciones.xlsx', sheet_name = industry)
stages.rename(columns = {'HT6 Code':'cmdCode'}, inplace = True)
stages["cmdCode"] = stages["cmdCode"].astype('str')
base["cmdCode"] = base["cmdCode"].astype('str')

#Vlook up of phases provided and main base with HT6 Codes
final_df  = pd.merge(base, stages, on ='cmdCode', how ='left')
final_df = final_df[["pfCode", "period", "aggrLevel", "rgDesc", "rtTitle", "ptTitle", "cmdCode",
                     "cmdDescE", "Naic_descrip", "TradeQuantity", "TradeValue", "PHASE", "COMPLEXITY"]]
final_df.rename(columns = {'rtTitle':'From', 'ptTitle':'To'}, inplace = True)
final_df

,pfCode,period,aggrLevel,rgDesc,From,To,cmdCode,cmdDescE,Naic_descrip,TradeQuantity,TradeValue,PHASE,COMPLEXITY
0,H4,2020,6,Export,Aruba,"Areas, nes",253090,Mineral substances; n.e.c. in chapter 25,"MINERAL SUBSTANCES, NESOI",64,100,Upstring,NaN
1,H4,2020,6,Export,Aruba,"Areas, nes",850760,"Electric accumulators; lithium-ion, including ...",LITHIUM ION BATTERIES,0,975,Downstream,NaN
2,H4,2020,6,Export,Aruba,"Areas, nes",850780,"Electric accumulators; other than lead-acid, n...",STORAGE BATTERIES NESOI,0,731,Downstream,NaN
3,H5,2020,6,Export,Australia,Armenia,253090,Mineral substances; n.e.c. in chapter 25,"MINERAL SUBSTANCES, NESOI",89,6966,Upstring,NaN
4,H5,2020,6,Export,Austria,Albania,253090,Mineral substances; n.e.c. in chapter 25,"MINERAL SUBSTANCES, NESOI",18000,6109,Upstring,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
33031,H5,2020,6,Export,Zambia,Zimbabwe,850720,"Electric accumulators; lead-acid, (other than ...",LEAD-ACID STORAGE BATTERIES NESOI,2533,190610,Downstream,NaN
33032,H5,2020,6,Export,Zambia,Zimbabwe,850760,"Electric accumulators; lithium-ion, including ...",LITHIUM ION BATTERIES,0,14598,Downstream,NaN
33033,H5,2020,6,Export,Zimbabwe,Zambia,850780,"Electric accumulators; other than lead-acid, n...",STORAGE BATTERIES NESOI,0,15792,Downstream,NaN
33034,H5,2020,6,Export,Zambia,Zimbabwe,850780,"Electric accumulators; other than lead-acid, n...",STORAGE BATTERIES NESOI,0,17822,Downstream,NaN


### Correct that every code has the same description

In [23]:
grouped_df = final_df.groupby("cmdCode")
first_values = grouped_df.first()
first_values = first_values.reset_index()
first_values = first_values[['cmdCode', 'cmdDescE']]
first_values.rename(columns = {'cmdDescE':'des'}, inplace = True)
corrected_df  = pd.merge(final_df, first_values, on ='cmdCode', how ='left')
del corrected_df['cmdDescE']
corrected_df.rename(columns = {'des':'cmdDescE'}, inplace = True)

### Construct indicator that a relation between countries is repeated for one product

In [24]:
corrected_df["max"] = corrected_df[["From", "To"]].max(axis=1)
corrected_df["min"] =  corrected_df[["From", "To"]].min(axis=1)
corrected_df["cmdCode"] = corrected_df["cmdCode"].astype('str')
corrected_df = corrected_df.sort_values(by=["max", "min", "cmdCode"]).reset_index(drop=True)
duplicate_df = corrected_df[["max", "min", "cmdCode"]]
duplicate_df = duplicate_df.assign(is_duplicate= lambda d: d.duplicated()).sort_values(["max", "min", "cmdCode"]).reset_index(drop=True)
corrected_df["is_duplicate"] = duplicate_df["is_duplicate"].astype(int)
corrected_df["is_duplicated"]= np.where(corrected_df["is_duplicate"]==0, '#00FF00', '#FF0000')

### Add Balassa Index

In [25]:
final_df = corrected_df

def balassa_index(final_df):
    
    data_empty = []
    total_exp = pd.DataFrame(data_empty, columns=['pfCode', 'period', 'aggrLevel', 'rgDesc', 'rtTitle', 'ptTitle',
                       'cmdCode', 'cmdDescE', 'TradeQuantity', 'TradeValue'])
    
    url = "http://comtrade.un.org/api/get?max=502&type=C&freq=A&px=HS&ps=2020&r=all&p=0&rg=2&cc=TOTAL"
    print(url)
    row = query(url)
    
    #Country's i total export of good k to the World
    exp_good_country = final_df.groupby(["From", "cmdCode"]).sum().reset_index()[["From", "cmdCode", "TradeValue"]]
    exp_good_country.rename(columns = {"TradeValue":'exp_good_country'}, inplace = True)
    final_df = pd.merge(final_df, exp_good_country, on = ["From", "cmdCode"] , how ='left')
    
    #Country's total exports to the world
    total_exp_country = pd.concat([total_exp, row])
    total_exp_country = total_exp_country[["rtTitle", "TradeValue", "period"]]
    total_exp_country.rename(columns = {"TradeValue":'total_exp_country', "rtTitle":'From'}, inplace = True)
    final_df = pd.merge(final_df, total_exp_country[["From","total_exp_country"]], on ='From', how ='left')
    
    #World export of good k
    exp_good = final_df.groupby(["cmdCode"]).sum().reset_index()[["cmdCode", "TradeValue"]]
    exp_good.rename(columns = {"TradeValue":'exp_good'}, inplace = True)
    final_df = pd.merge(final_df, exp_good, on ='cmdCode', how ='left')
    
    #World total exports
    total_exp = total_exp_country.groupby(["period"]).sum().reset_index()[["total_exp_country"]]
    final_df["total_exp"] = int(total_exp['total_exp_country'].values[0])
    
    #Balassa index
    final_df["balassa"] = (final_df["exp_good_country"] / final_df["total_exp_country"]) / (final_df["exp_good"] / final_df["total_exp"])
    return final_df 

final_df = balassa_index(final_df)
final_df = final_df[['pfCode', 'period', 'aggrLevel', 'rgDesc', 'From', 'To', 'cmdCode',
       'Naic_descrip', 'TradeQuantity', 'TradeValue', 'PHASE', 'COMPLEXITY',
       'max', 'min', 'cmdDescE', 'is_duplicated','balassa', 'is_duplicate']]

http://comtrade.un.org/api/get?max=502&type=C&freq=A&px=HS&ps=2020&r=all&p=0&rg=2&cc=TOTAL


### Add broader description of products for semiconductors

In [26]:
string = industry + '_traduccion.csv'
broad = pd.read_csv(string)[['naics_6', 'naics_description', 'cmdCode']]
broad["cmdCode"] = broad["cmdCode"].astype('str')
final_df["cmdCode"] = final_df["cmdCode"].astype('str')
broad = broad[broad['cmdCode'].isin(final_df['cmdCode'].unique().tolist())]
final_df = pd.merge(final_df, broad, on ='cmdCode', how ='left')
string = 'db_trade_f_' + industry + '.csv '
final_df.to_csv (string, index = False, header=True)

### Add proximity index

Calculate total of countries to take into account for the index and create empty structures to store information.

In [27]:
url = "http://comtrade.un.org/api/get?max=502&type=C&freq=A&px=HS&ps=2020&r=all&p=0&rg=2&cc=TOTAL"
print(url)
result_query = query_v2(url)
countries = result_query['rtCode'].unique().tolist()
empty = []
all_products = pd.DataFrame(empty, columns=['pfCode', 'period', 'aggrLevel', 'rgDesc', 'rtTitle', 'ptTitle',
                       'cmdCode', 'cmdDescE', 'TradeQuantity', 'TradeValue'])
lst_countries = []

http://comtrade.un.org/api/get?max=502&type=C&freq=A&px=HS&ps=2020&r=all&p=0&rg=2&cc=TOTAL


For loop to get data of all products that every country exports. (repeat until you get all de countries)

In [69]:
for c in countries:
    if c not in lst_countries:
        url = "http://comtrade.un.org/api/get?max=100000&type=C&freq=A&px=HS&ps=2020&r=" + str(c) + "&p=0&rg=2&cc=ALL"
        print(url)
        row = query(url)
        all_products = pd.concat([all_products, row])
        lst_countries.append(c)

http://comtrade.un.org/api/get?max=100000&type=C&freq=A&px=HS&ps=2020&r=8&p=0&rg=2&cc=ALL
http://comtrade.un.org/api/get?max=100000&type=C&freq=A&px=HS&ps=2020&r=31&p=0&rg=2&cc=ALL
http://comtrade.un.org/api/get?max=100000&type=C&freq=A&px=HS&ps=2020&r=32&p=0&rg=2&cc=ALL
http://comtrade.un.org/api/get?max=100000&type=C&freq=A&px=HS&ps=2020&r=36&p=0&rg=2&cc=ALL
http://comtrade.un.org/api/get?max=100000&type=C&freq=A&px=HS&ps=2020&r=40&p=0&rg=2&cc=ALL
http://comtrade.un.org/api/get?max=100000&type=C&freq=A&px=HS&ps=2020&r=51&p=0&rg=2&cc=ALL
http://comtrade.un.org/api/get?max=100000&type=C&freq=A&px=HS&ps=2020&r=52&p=0&rg=2&cc=ALL
http://comtrade.un.org/api/get?max=100000&type=C&freq=A&px=HS&ps=2020&r=56&p=0&rg=2&cc=ALL
http://comtrade.un.org/api/get?max=100000&type=C&freq=A&px=HS&ps=2020&r=60&p=0&rg=2&cc=ALL
http://comtrade.un.org/api/get?max=100000&type=C&freq=A&px=HS&ps=2020&r=68&p=0&rg=2&cc=ALL
http://comtrade.un.org/api/get?max=100000&type=C&freq=A&px=HS&ps=2020&r=70&p=0&rg=2&cc=ALL


http://comtrade.un.org/api/get?max=100000&type=C&freq=A&px=HS&ps=2020&r=484&p=0&rg=2&cc=ALL
http://comtrade.un.org/api/get?max=100000&type=C&freq=A&px=HS&ps=2020&r=533&p=0&rg=2&cc=ALL
http://comtrade.un.org/api/get?max=100000&type=C&freq=A&px=HS&ps=2020&r=818&p=0&rg=2&cc=ALL


Know codes for the industry

In [71]:
HT6_industry = pd.read_excel (r'Descripciones.xlsx', sheet_name= industry)
industry_codes = HT6_industry['HT6 Code'].unique().tolist()
industry_codes = list(map(str, industry_codes))

Add balassa to all products

In [72]:
all_products.rename(columns = {"rtTitle":'From'}, inplace = True)
all_products = all_products[all_products['cmdCode'] != 'TOTAL']
all_products[["cmdCode", "period", "aggrLevel", "TradeQuantity", "TradeValue"]] = all_products[["cmdCode", "period", "aggrLevel", "TradeQuantity", "TradeValue"]].apply(pd.to_numeric)
all_products_balassa = balassa_index(all_products)
all_products_balassa = all_products_balassa[['pfCode', 'period', 'aggrLevel', 'rgDesc', 'From', 'ptTitle', 'cmdCode',
       'TradeQuantity', 'TradeValue', 'cmdDescE','balassa']]
all_products_balassa = all_products_balassa[all_products_balassa['balassa'] >= 1]

http://comtrade.un.org/api/get?max=502&type=C&freq=A&px=HS&ps=2020&r=all&p=0&rg=2&cc=TOTAL


### Construct proximity index

In [73]:
#Calculate countries that export both

countries_industry = all_products_balassa[all_products_balassa['cmdCode'] == main_ht6_code][['From']]
countries_industry['yes_no'] = 1
countries_industry = pd.merge(all_products_balassa, countries_industry, on ='From', how ='left')
countries_industry['yes_no'] = countries_industry['yes_no'].fillna(0)
join_products = countries_industry.groupby(['cmdDescE', 'cmdCode', 'yes_no']).count().reset_index()[['pfCode', 'cmdCode', 'yes_no']]
join_products.rename(columns = {"pfCode":'count_join'}, inplace = True)
join_products = join_products[join_products['yes_no'] == 1]


#Calculate countries that export each product

count_products = all_products_balassa.groupby(['cmdDescE', 'cmdCode']).count().reset_index()[['pfCode', 'cmdCode', 'cmdDescE']]
count_products.rename(columns = {"pfCode":'count_countries'}, inplace = True)


#Calculate proximity indicator by using the past 2 measures
final_df_merge = pd.merge(count_products, join_products, on ='cmdCode', how ='left')
final_df_merge['industry_count'] = len(all_products_balassa[all_products_balassa['cmdCode'] == main_ht6_code]['From'].unique().tolist())
final_df_merge = final_df_merge[final_df_merge['yes_no'] == 1]
final_df_merge['max'] = final_df_merge[["industry_count", "count_countries"]].max(axis=1)
final_df_merge['proximity'] = final_df_merge['count_join'] / final_df_merge['max']
final_df_merge["cmdCode"] = final_df_merge["cmdCode"].astype(str)
final_df_merge['num_digits'] = final_df_merge['cmdCode'].str.len()
final_df_merge = final_df_merge[final_df_merge['num_digits'] == 6][['cmdCode', 'cmdDescE','proximity']]
final_df_merge = final_df_merge[~final_df_merge['cmdCode'].isin(industry_codes)][['cmdCode', 'cmdDescE','proximity']]
final_df_merge = final_df_merge.sort_values(by= 'proximity', ascending = False)

In [74]:
#Filter products in which Mexico is specialized
mexico_specialized = all_products_balassa[all_products_balassa['From'] == 'Mexico']['cmdCode'].unique().tolist()
mexico_specialized = [str(integer) for integer in mexico_specialized]
final_df_merge[final_df_merge['cmdCode'].isin(mexico_specialized)]
string = industry + '_proximity.csv'
final_df_merge.to_csv (string, index = False, header=True)

### Design documents for igraph

In [76]:
def prepare_edges_nodes(proximity_doc_string, number_of_nodes):

    #Nodes
    final_df_merge = pd.read_csv(proximity_doc_string)
    data_empty = pd.DataFrame([[main_ht6_code, str(industry), 1]], columns=['cmdCode', 'cmdDescE', 'proximity'])
    graph = pd.concat([data_empty,final_df_merge])                                                       
    graph = graph.sort_values(by= 'proximity', ascending = False)
    graph.insert(0, 'id', range(1, 1 + len(graph)))
    graph['Label'] = graph['cmdDescE'].apply(lambda x: x.split(';')[0])
    graph['Label'] = graph['Label'].apply(lambda x: x.split(',')[0])
    graph['proximity'] = graph['proximity'] * 10
    graph['proximity'] = graph['proximity'].apply(np.ceil)
    nodes = graph[['id', 'cmdCode', 'Label', 'proximity']].head(number_of_nodes)
    nodes_string = industry + '_nodes.csv'
    nodes.to_csv (nodes_string, index = False, header=True)
    
    #Edges
    edges = final_df_merge 
    edges.insert(0, 'from', 1)
    edges.insert(0, 'to', range(2, 2 + len(edges)))
    edges['weight'] = edges['proximity']*10
    edges['weight'] = edges['weight'].apply(np.ceil)
    edges = edges[['from', 'to', 'weight']].head(number_of_nodes - 1)
    edges_string = industry + '_edges.csv'
    edges.to_csv (edges_string, index = False, header=True)
    
prepare_edges_nodes(string, 28)